#### LengthBasedExampleSelector

Dynamic 하게 예제를 고르는 방법을 학습합니다. 

왜냐하면, 때로는 정말 많은 예제들이 있을 수 있고 그렇게 되면 어느 정도의 example 들을 골라서 Prompt에 허용할건지 정해야 합니다.

이유는 많은 prompt는 더 많은 비용을 지불해야 하기 때문입니다.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

In [3]:
chat = ChatOpenAI(
    temperature=0.1,        # 창의성 (0 ~ 2)
    model='gpt-3.5-turbo',  # 사용 모델 지정 (Default : gpt-3.5-turbo)
    streaming=True,         # Streaming ON
    callbacks=[StreamingStdOutCallbackHandler()]
)

C:\Users\SangHyeonLee\AppData\Local\Temp\ipykernel_20236\579124313.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [4]:
# 랜덤 예제 선택기 생성하기
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [5]:
# FewShot 예제
examples = [
{
    "question": "프랑스에 대해 무엇을 알고 있어?",
    "answer": """
    제가 아는 것은 다음과 같습니다:
    - 수도: 파리
    - 언어: 프랑스어
    - 음식: 와인과 치즈
    - 통화: 유로
""",
},
{
    "question": "이탈리아에 대해 무엇을 알고 있어?",
    "answer": """
    저는 이것을 알고 있습니다:
    - 수도: 로마
    - 언어: 이탈리아어
    - 음식: 피자와 파스타
    - 통화: 유로
    """,
},
{
    "question": "그리스에 대해 무엇을 알고 있어?",
    "answer": """
    저는 이것을 알고 있습니다:
    - 수도: 아테네
    - 언어: 그리스어
    - 음식: 수블라키와 페타 치즈
    - 통화: 유로
    """,
    },
]

# 1. 예제의 형식을 지정
example_template = """
    Human: {question}
    AI : {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

# example_selector = LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_lengt=10                    # 예제의 양을 지정할 수 있습니다.
# )

example_selector = RandomExampleSelector(
    examples=examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: {country}에 대해 무엇을 알고 있어?"  ,                      # 형식화된 모든 예제 마지막에 나는 내용
    input_variables=['country']                                             # 유효성 검사
)

prompt.format(country='korea')

'\n    Human: 이탈리아에 대해 무엇을 알고 있어?\n    AI : \n    저는 이것을 알고 있습니다:\n    - 수도: 로마\n    - 언어: 이탈리아어\n    - 음식: 피자와 파스타\n    - 통화: 유로\n    \n\n\nHuman: korea에 대해 무엇을 알고 있어?'